In [ ]:
#V2 is onlyunstructured, also it changes the logic on how to do the match ACCT and PET

In [1]:
from os.path import join
from datetime import time, datetime
import os
import glob
import numpy as np
import SimpleITK as sitk
import pydicom as dicom
from pydicom.tag import Tag
import sys
import csv
import pydicom as pdcm

In [2]:
def getICUID(path):
    for root, dirs, files in os.walk(path):
        for f in files:
            slices = pdcm.read_file(root+f,stop_before_pixels=True)
            return slices.file_meta.ImplementationClassUID

In [3]:
window_type = 'lung'
root_path="//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/"
structured = root_path + "DATA_VOLLEDIG_structured/"
unstructured = root_path + "DATA_VOLLEDIG_unstructured/"
csv_filename = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/ListPETMatchACCT_V2_Try3.csv"

#Folders to look at:
#Try1 ="CT_DICOM_CLEAN/"
Try2 ="PET_DICOM/"
#Try3 ="PET_LDCT_REGISTERED/"
#Try4 ="PET_MIRADA_REGISTERED/"

In [12]:
MergeLists=[]
#MergeLists.append(os.path.join(structured,Try1))
MergeLists.append(os.path.join(structured,Try2))
#MergeLists.append(os.path.join(structured,Try3))
#MergeLists.append(os.path.join(structured,Try4))
MergeLists.append(os.path.join(unstructured))

for currdir in MergeLists:
    print(os.path.exists(currdir))

True
True


In [13]:
modalityToLook = ["CT","PET"]

for currdir in MergeLists:
    PxList = np.sort(os.listdir(currdir))
    print("Number of Patients:",len(PxList)," Example: ",PxList[1])
 

Number of Patients: 801  Example:  0010539
Number of Patients: 1285  Example:  0004879


In [14]:
MergeLists[1]

'//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/DICOM_data/DATA_VOLLEDIG_unstructured/'

In [17]:

PxList = np.sort(os.listdir(MergeLists[1]))


dict_PETperPx = {}
PxlistwPETS = []
for Px in PxList:
    matchingPETs = []
    for root, dirs, files in os.walk(MergeLists[1]+Px, topdown=True):
        ACCT_LD_fpaths = []
        PET_fpaths = []
        
        bool_PET = False
        bool_ACCT = False
        bool_match = False

        for d in dirs:
            if "CT" in d:
                ACCT_LD_fpaths.append(root+'/'+d+'/')
                bool_ACCT = True
            if "PET" in d and not("CT" in d):
                PET_fpaths.append(root+'/'+d+'/')
                bool_PET = True
        if bool_PET:
            for currPET in PET_fpaths:
                for currCT in ACCT_LD_fpaths:
                    lastUID_CT = getICUID(currCT)
                    lastUID_PET = getICUID(currPET)
                    if lastUID_CT == lastUID_PET:
                        bool_match = True 
                        matchingPETs.append([Px,currCT,currPET])
                        PxlistwPETS.append([Px,currCT,currPET])
        else:
            dict_PETperPx[Px]="Missing"
                        
    if len(matchingPETs)>0:
        for i in range(len(matchingPETs)):
            petname = matchingPETs[i][2].split("/")[-2]
            dict_PETperPx[Px]=petname

In [18]:
#SavePaths
filename = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/ListPETMatchACCT_V2_WTry3_pt2.csv"
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(PxlistwPETS)

In [25]:
#Save Dictionary, but not ideal, because only last PET is stored, for those that have many 
#useful to see which doesnt have a pet
filename = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/CSVs/DictionatyPETMatchACCT_V2WTry2.csv"
with open(filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["Key", "Value"])
    writer.writeheader()
    for key, value in dict_PETperPx.items():
        writer.writerow({"Key": key, "Value": value})